In [1]:
!pip install geopandas
!pip install mgwr
!pip install numba

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 37.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 28.7 MB/s eta 0:00:0000:0100:01
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 53.6 MB/s eta 0:00:00
  Created wheel for mgwr: filename=mgwr-2.1.2-py3-none-any.whl size=46367 sha256=64762d02f40cda4e19e5109d8ea9dca16f83d81f9c6777257fda57cea7c0

In [38]:
import numpy as np
import pandas as pd
import pickle
import os
import multiprocessing
os.environ['USE_PYGEOS'] = '0'
import geopandas as gp
import multiprocessing as mp
from mgwr.gwr import GWR,MGWR
from mgwr.sel_bw import Sel_BW

In [39]:
#Load the funda data
funda_data = gp.read_file("data/Houseprices/funda_buy_amsterdam_31-03-2023_full_distances.gpkg")

# funda_data = funda_data.loc[funda_data['price'] < 2500000]
# funda_data = pd.DataFrame(funda_data)

# funda_data = gp.GeoDataFrame(pd.get_dummies(funda_data, columns = ["house_type", "building_type", "energy_label", "has_balcony", "has_garden"]))

In [40]:
#show funda data
funda_data


,X,house_id,house_type,building_type,price,price_m2,room,bedroom,bathroom,living_area,...,metro_dist,train_dist,school_dist,shops_dist,centre_dist,zuid_dist,ndvi100,ndvi300,ndvi500,geometry
0,18,42082895,appartement,Resale property,675000,7500.0,3,2,1,90,...,668.095770,1290.089283,153.948872,151.069423,2391.694363,2455.223172,0.162001,0.196801,0.195157,POINT (121858.014 484983.965)
1,51,42057103,appartement,Resale property,1295000,6475.0,5,3,0,200,...,373.221221,1985.945869,313.440368,110.727430,1788.727768,2560.149839,0.142814,0.184581,0.184497,POINT (121381.001 485541.002)
2,79,42038931,appartement,Resale property,385000,7549.0,4,3,0,51,...,576.663789,1378.133276,130.618369,387.194432,2546.495664,2210.323492,0.233371,0.195568,0.208979,POINT (121691.846 484800.990)
3,107,42005049,appartement,Resale property,650000,7738.1,4,3,1,84,...,689.699113,1524.552850,260.958705,100.917074,2003.207149,2709.243910,0.183670,0.184798,0.169328,POINT (121834.000 485376.000)
4,212,42165208,appartement,Resale property,465000,8157.9,3,2,1,57,...,778.780657,1085.901160,180.489983,112.154131,2594.803303,2462.246204,0.215336,0.204926,0.200075,POINT (122003.041 484808.014)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,524,88454958,appartement,Resale property,289000,3524.4,3,2,1,82,...,304.495796,1894.684593,48.763585,754.328744,9220.212311,7952.978298,0.303762,0.319314,0.291643,POINT (127290.990 480243.999)
1715,527,88460312,appartement,Resale property,335000,3764.0,4,3,1,89,...,465.786814,1740.296080,29.809326,818.953743,9167.566064,7835.367130,0.287997,0.326125,0.296571,POINT (127136.997 480186.005)
1716,1009,42065127,appartement,Resale property,299000,4271.4,3,2,1,70,...,486.295154,1719.114500,33.504192,834.294781,9169.880396,7827.847089,0.303994,0.323947,0.300770,POINT (127121.997 480171.002)
1717,1460,42912098,appartement,Resale property,280000,4058.0,3,2,1,69,...,166.117549,2034.389126,184.375679,620.397055,9219.168740,8015.276792,0.367509,0.294422,0.296814,POINT (127397.001 480335.000)


In [58]:
#create array with the dependent variable
b_y = funda_data['price_m2'].values.reshape((-1,1))

In [59]:
# #create an array with the indepentend variables (order matters for the extraction of params later)
# b_X = funda_data[['room', 'bedroom', 'bathroom', 'living_area',
#                   'house_age', 'bus_dist', 'subway_dist', 'train_dist',
#                   'university_dist', 'school_dist', 'mall_dist', 'supermarket_dist',
#                   'energy_label_A', 'energy_label_B', 'energy_label_C', 'energy_label_D',
#                   'energy_label_E', 'energy_label_F', 'energy_label_G', 'energy_label_na',
#                   'has_balcony_0.0', 'has_balcony_1.0', 'has_garden_0.0', 'has_garden_1.0']].values 

# cols = ['room', 'bedroom', 'bathroom', 'living_area',
#                   'house_age', 'train_dist', 'metro_dist', 'tram_dist', 'ndvi100']

cols = ['living_area','room', 'bedroom', 'bathroom',
                  'house_age', 'train_dist', 'metro_dist', 'tram_dist', 'ndvi500', 'centre_dist', 'zuid_dist', 'shops_dist', 'school_dist']

b_X = funda_data[cols].values 

In [60]:
#create coordinate tuple for the model
u = funda_data['geometry'].x
v = funda_data['geometry'].y
b_coords = list(zip(u, v))

In [61]:
multiprocessing.cpu_count()

8

In [62]:
#This might be needed to turn off the OpenMP multi-threading
%env OMP_NUM_THREADS = 1

env: OMP_NUM_THREADS=1


In [63]:
#Parrallelization is more favored when you your data are large and/or your machine have many many cores.
#mgwr has soft dependency of numba, please install numba if you need better performance (pip install numba).
n_proc = 4 #two processors
pool = mp.Pool(n_proc) 

In [64]:
%%time
#Run basic GWR in parrallel mode

gwr_selector = Sel_BW(b_coords, b_y, b_X)
gwr_bw = gwr_selector.search(pool = pool) #add pool to Sel_BW.search
print(gwr_bw)
gwr_results = GWR(b_coords, b_y, b_X, gwr_bw).fit(pool = pool)

205.0
CPU times: total: 141 ms
Wall time: 9.38 s


In [65]:
#show summary
gwr_results.summary()

Model type                                                         Gaussian
Number of observations:                                                1719
Number of covariates:                                                    14

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                       4967007514.484
Log-likelihood:                                                  -15225.580
AIC:                                                              30479.161
AICc:                                                             30481.443
BIC:                                                           4966994813.090
R2:                                                                   0.475
Adj. R2:                                                              0.471

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- --

In [13]:
%%time
#run MGWR in parrallel mode. Note: max_iter_multi needs to be specified

mgwr_selector = Sel_BW(b_coords, b_y, b_X, multi=True)
mgwr_bw = mgwr_selector.search(pool=pool, max_iter_multi=200, criterion = "AICc", verbose = True) #add pool to Sel_BW.search
print(mgwr_bw)


Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

Current iteration: 1 ,SOC: 0.0065626999999999994
Bandwidths: 43.0, 43.0, 43.0, 89.0, 60.0, 43.0, 43.0, 43.0, 43.0
Current iteration: 2 ,SOC: 0.0036994
Bandwidths: 43.0, 51.0, 59.0, 99.0, 58.0, 43.0, 52.0, 46.0, 43.0
Current iteration: 3 ,SOC: 0.002451
Bandwidths: 44.0, 71.0, 70.0, 133.0, 60.0, 43.0, 67.0, 51.0, 43.0
Current iteration: 4 ,SOC: 0.0020494
Bandwidths: 46.0, 79.0, 88.0, 175.0, 63.0, 51.0, 72.0, 59.0, 46.0
Current iteration: 5 ,SOC: 0.0018195
Bandwidths: 50.0, 93.0, 106.0, 193.0, 69.0, 52.0, 97.0, 59.0, 50.0
Current iteration: 6 ,SOC: 0.0018621
Bandwidths: 78.0, 112.0, 126.0, 222.0, 74.0, 52.0, 117.0, 59.0, 52.0
Current iteration: 7 ,SOC: 0.0016062
Bandwidths: 86.0, 121.0, 136.0, 230.0, 86.0, 52.0, 136.0, 59.0, 52.0
Current iteration: 8 ,SOC: 0.0014655999999999998
Bandwidths: 91.0, 125.0, 145.0, 283.0, 92.0, 52.0, 152.0, 52.0, 65.0
Current iteration: 9 ,SOC: 0.0014202
Bandwidths: 93.0, 145.0, 164.0, 351.0, 103.0, 52.0, 174.0, 47.0, 72.0
Current iteration: 10 ,SOC: 0.0013349


In [16]:
#pickle.dump(mgwr_selector, open('data/models/mgwr_selector.pkl', 'wb'))

In [32]:
#mgwr_selector = pickle.load(open('data/models/mgwr_selector.pkl', 'rb'))

In [33]:
mgwr_results = MGWR(b_coords, b_y, b_X, selector=mgwr_selector).fit(pool = pool)


C:\Users\jitse\anaconda3\lib\site-packages\mgwr\gwr.py:1578: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  #progress bar


Inference:   0%|          | 0/4 [00:00<?, ?it/s]

In [34]:
#show MGWR model summary
mgwr_results.summary()

Model type                                                         Gaussian
Number of observations:                                                1719
Number of covariates:                                                     9

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                       6830063003.120
Log-likelihood:                                                  -15499.345
AIC:                                                              31016.691
AICc:                                                             31018.819
BIC:                                                           6830050264.478
R2:                                                                   0.279
Adj. R2:                                                              0.275

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- --

In [66]:
#recreate R format table

def return_geopackage(mgwr_results,cols, path_name):
    df = gp.GeoDataFrame()
    df['Intercept'] = mgwr_results.params[:,0]
    df['intercept_SE'] = mgwr_results.bse[:,0]
    df['intercept_TV'] = mgwr_results.tvalues[:,0]
    
    df['yhat'] = mgwr_results.predy
    df['residual'] = mgwr_results.y.reshape((-1,1)) - mgwr_results.predy
    
    for i,col in enumerate(cols):
        
        df[col] = mgwr_results.params[:,i+1]

        df[col+'_SE'] = mgwr_results.bse[:,i+1]
        df[col+'_TV'] = mgwr_results.tvalues[:,i+1]
        
    df['geometry'] = funda_data['geometry']
    df.to_file(path_name)

In [67]:
#show the GeoDataFrame and write it to a folder
return_geopackage(gwr_results,cols, 'data/models/gwr_results_amsterdam_ndvi_m2.gpkg')

In [68]:
#pickle.dump(gwr_results, open('data/models/gwr_results_amsterdam_ndvi_m2.pkl', 'wb'))

In [26]:
#gwr_results = pickle.load(open('data/models/mgwr_results.pkl', 'rb'))